In [13]:
import socket
import hashlib
import random
import string
from struct import pack, unpack

In [14]:
def randStr():
    return ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(16))

In [19]:
def findNonce(inp):
    while True:
        st = randStr()
        te = inp + st
        te = te.encode("utf-8")
        ans = hashlib.sha256(te).hexdigest()
        if ans[-1] in ['0', '1', '2', '3', '4']:
            return st

In [22]:
# print("This is stub")
procNo = 0
balance = 100
# tes = hashlib.sha256("nihao".encode("utf-8")).hexdigest()
# print(findNonce(tes))
ports = [10001, 10002, 10003, 10004, 10005]
linkSuc = [False] * 5

[False, False, False, False, False]

In [15]:
# randStr()

'YZFG9PLRTWJYWS80'